In [2]:
import os, sys

base_path = os.path.join(os.getcwd(), "..")
print(f"Base Path: {base_path}")
sys.path.append(base_path)

Base Path: /Users/swkim/Documents/coding/thesis/kube_scheduler_master2/notebook/..


# Data Generation

## 1. 1-label data generation

In [2]:
# Load gym environment
import gym
from kube_sim_gym import *
from kube_sim_gym.envs.sim_kube_env import SimKubeEnv

In [3]:
from kube_hr_scheduler.scheduler.sim_hr_scheduler import SimHrScheduler
from kube_hr_scheduler.strategies.model.default import Model

In [4]:
env = gym.make('SimKubeEnv-v0')
scheduler = SimHrScheduler(env, 'default.py')

In [ ]:
# Data generation
# Data consists of state, scheduler decision
data_size = 1000000
data_path = os.path.join(base_path, "dataset", "data_1.csv")
with open(data_path, 'w') as f:
    for i in range(data_size):
        state = list(env.random_state_gen())
        action = scheduler.decision(env)
        data = state + [action]
        f.write(','.join(list(map(str,data))) + '\n')

## 2. 6-confidence data generation

In [7]:
# Load gym environment
import gym
from kube_sim_gym import *
from kube_sim_gym.envs.sim_kube_env import SimKubeEnv

In [8]:
from kube_hr_scheduler.scheduler.sim_hr_scheduler import SimHrScheduler
from kube_hr_scheduler.strategies.model.default import Model

In [9]:
env = gym.make('SimKubeEnv-v0')
scheduler = SimHrScheduler(env, 'default.py')

In [ ]:
# Data generation
# Data consists of state, scheduler decision
data_size = 1000000
data_path = os.path.join(base_path, "dataset", "data_5.csv")
with open(data_path, 'w') as f:
    for i in range(data_size):
        state = list(env.random_state_gen())
        confidence = scheduler.model.get_confidence(env)
        data = state + confidence.tolist()
        f.write(','.join(list(map(str,data))) + '\n')

## 3. 1-confidence data generation

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from kube_mm_scheduler.model.net3 import Net3

net3 = Net3()

In [4]:
# predict the score for each action
def predict_score(data, model=net3):
    model.eval()
    if not torch.is_tensor(data):
        data = torch.tensor(data, dtype=torch.float32)
    data10 = data[:, :-2]
    data2 = data[:, -2:]
    output = model(data10, data2)
    # Round up at the 4th decimal point
    output = torch.round(output * 10000) / 10000
    # # Softmax
    # output = F.softmax(output, dim=1)
    # print(output)
    return output

In [5]:
# Read from data_1.csv and formulate data_3.csv which has additional selected node information
src_path = os.path.join(base_path, "dataset", "data_1.csv")
with open(src_path, 'r') as f:
    data_path = os.path.join(base_path, "dataset", "data_3.csv")
    with open(data_path, 'w') as g:
        for line in f.readlines():
            line = line.strip()
            line = line.split(',')
            node_state = line[:-3]
            pod_quota = line[-3:-1]

            # Confidence
            flt_state = list(map(float, node_state+pod_quota))
            # print(flt_state)
            confidence = predict_score([flt_state])

            for i in range(5):
                selected_node_state = node_state[i*2:i*2+2]
                new_state = node_state + pod_quota + selected_node_state
                conf = confidence[0].tolist()[i+1]
                # Takes the 4th decimal point
                conf = round(conf, 4)
                
                new_data = new_state + [conf]
                g.write(','.join(list(map(str,new_data))) + '\n')

KeyboardInterrupt: 